## CD-HIT clustering

CD-HIT expects data as FASTA format and output them in a specific format (we use `cdhit_reader` to parse it)

In [5]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import gzip
from random import shuffle

## FASTA input

In [6]:
MINLENGTH = 70
MAXLENGTH = 1000
# for most proteins uniprotEnd == Length but there are rare exceptions in knotted d.f.

knotted = pd.read_csv("spout_all_knotted.csv.gz").query(f"uniprotEnd >= {MINLENGTH}").query(f"uniprotEnd <= {MAXLENGTH}")
unknotted = pd.read_csv("spout_all_unknotted.csv.gz").query(f"uniprotEnd >= {MINLENGTH}").query(f"uniprotEnd <= {MAXLENGTH}")

In [3]:
unknotted

,ID,latestVersion,globalMetricValue,uniprotStart,uniprotEnd,uniprotSequence,Length
0,AF-A0A009NLX4-F1,4,94.56,1,306,MALRHFLTLRDLSTLELNRILERASELKKMQHDNKVYQPFVGKVLG...,306
1,AF-A0A010QHM8-F1,4,74.75,1,575,MSQLRDEKDNTTLNDSKESTNPKVVVDSVFDTSEKLFLGGIDDGSD...,575
2,AF-A0A010R445-F1,4,91.31,1,353,MKQTAFRVTRQALNGAQSRAYSQSTGPRHLMSIADLSPAEFATLVR...,353
3,AF-A0A014C010-F1,4,94.69,1,306,MALRHFLTLRDLSTLELNRVLQRASELKKMQQSNKVYQPFVGKVLG...,306
4,AF-A0A014CD17-F1,4,94.38,1,306,MALRHFLTLRDLSTLELNRVLQRASELKKMQQSNKVYQPFVGKVLG...,306
...,...,...,...,...,...,...,...
331047,AF-X5X2N4-F1,4,76.69,1,632,MAGPGDNTRNKSKTGSEADSFKRAVTVCMRAIAGDKELEVGFAKDR...,632
331048,AF-X6E033-F1,4,90.69,1,492,MRNYRGLVHAVGGFAGDRSGNFAVLFGFAASVLALAAGFSVNITQL...,492
331049,AF-X6M7S0-F1,4,90.56,1,355,MSFFKRKIPQLFWNQLRNVSSTRTTKHVLKISDLSQKELRDVLTFA...,355
331050,AF-X7ZDJ7-F1,4,76.44,1,708,MGRHSLPDPEDSADEPPDEYAAEQQDWADQIADQPGGGRHSEVGYP...,708


In [4]:
knotted_records = [SeqRecord(Seq(seq), id=ID, description="knotted") for seq, ID in zip(knotted['uniprotSequence'], knotted['ID'])]
unknotted_records = [SeqRecord(Seq(seq), id=ID, description="unknotted") for seq, ID in zip(unknotted['uniprotSequence'], unknotted['ID'])]

In [5]:
import random

records = knotted_records + unknotted_records
shuffle(records)

SeqIO.write(records, "all.fasta", "fasta")

759683

## 2) CD-HIT

In [ ]:
# Takes long time, better run it in the terminal
#!cd-hit -i all.fasta -o tight_clusters -c 0.97 -s 0.95 -G 0 -aS 0.9 -T 0
# CD-HIT version 4.8.1 (built on Aug 20 2021)

## 3) Parsing CD-HIT output

In [1]:
from cdhit_reader import read_cdhit

In [2]:
clusters = read_cdhit("./tight_clusters.clstr").read_items()

In [7]:
knotted = knotted.set_index("ID")
unknotted = unknotted.set_index("ID")

##  4) Search for non-trivial cluster (= both knotted and unknotted proteins), minimal edit distance

Work in progress

In [8]:
nontrivial = [cl for cl in clusters if len(cl.sequences)>1]

In [9]:
len(nontrivial)

97327

In [10]:
from tqdm.notebook import tqdm

In [31]:
both_knotted_and_unknotted = []

for cl in tqdm(nontrivial):
    if not (all([x.name in knotted.index for x in cl.sequences]) or all([x.name in unknotted.index for x in cl.sequences])):
        both_knotted_and_unknotted.append(cl)

  0%|          | 0/97327 [00:00<?, ?it/s]

In [40]:
import pickle

with open("both_knotted_and_unknotted.pkl", "wb") as fw:
    pickle.dump(both_knotted_and_unknotted, fw)

len(both_knotted_and_unknotted)

2136

In [35]:
[(x.name, x.name in knotted.index) for x in both_knotted_and_unknotted[2].sequences]

[('AF-A0A5D2WZ99-F1', True),
 ('AF-A0A5D2WXX5-F1', True),
 ('AF-A0A5D2WXV2-F1', False)]

In [37]:
both_knotted_and_unknotted[2].sequences

[ClusterSequence(id=0, name=AF-A0A5D2WZ99-F1, length=996, identity=100.0, is_ref=True, seqtype=SeqType.PROTEIN, strand=Strand.NONE),
 ClusterSequence(id=1, name=AF-A0A5D2WXX5-F1, length=980, identity=98.39, is_ref=False, seqtype=SeqType.PROTEIN, strand=Strand.NONE),
 ClusterSequence(id=2, name=AF-A0A5D2WXV2-F1, length=955, identity=100.0, is_ref=False, seqtype=SeqType.PROTEIN, strand=Strand.NONE)]

In [38]:
knotted.loc['AF-A0A5D2WZ99-F1'].uniprotSequence

'MQRRKHGPKVQTKFEIQQLETEEQSSEAEASNIYNMFTDSLLLILEELVSFCNLSCSIFSSGSDIEDKVLPSSVRGKLGGPSQRRLSISLTTAVLQATISVKAVAYIYAWCAQHRTGILLNSAFTAVWKFFCNTITSPTCNSESEAEVRIAAYEALAPALKALVSVLAPQILVLFMENSKSLQPAIEGEPWLDTLVLSFLQNINNLLAVGFMVRTRRAVLLNWKWVILESLLSIPSYSFGSKLQQEDGSFFFSDAALINIFTDTVESLENAGEGSVLPILRSIRLALELFASRRLSLVVSRFTGVDSQMIWRLVHSSWILHSSCNKRRVAHIAALLSSVLHPSLFSDGDMHDADNEPGPLKWFVEKLLEEGTKSPRTIRLAALHLTGLLLSNPTTIKYYIKELKLLTLYGSVAFDEDFEAELSENDDARTEVTLLTKIQDSELTEAFINTELYARVCVANLFYKLADLTNRSGSSTGDKASQSALESGKLFLLELLDTVLNDKELAKELYKKYSAIHRRKIRAWQMICVLSQFVDDDIVEEVTQCLQIALYRNNLPSVRQYLETFAINIYMKFPSLVAEQLVPLLRDYDMRPQALSSYVFIAANVILHASKDIQLRHLDELLPPILPLLTSHHHSLRGFTQVLLHQVLRNFFPSLDSKSSEIIPLEKRCFEDLKLYLAKNSDCMRLRASMEGYLDAYNPKTCVTPAGIFVSRVEEIGFECVPTSLMEQVINFLNDVREDLRCSMAKDIVTIKNESLNMAAGSGSTEEVSSACEEKLELPKDAHLDFQKKITFSKHEKPDLGSSSLLCKGEVYKQLLEMEKEDDLLDQLWKSRSLAMERIRGNRQHIILVASLIDRIPNLAGLARTCEVFKTQGLAVADAKIVQDKQFQLISVTAEKWVPVIEVPVNSVKQFLEKKKREGFSILGLEQTANSVPLDQYIYPKKTVLVLGREKEGIPVDIIHVLDACIEIPQLGVVRSLNVHVSGAIALWEYTRQQRL'

In [22]:
unknotted.loc['AF-A0A671YMH3-F1'].uniprotSequence

'MRKGMAPLRGQKTGLQNVFYVCLSISVAYSYNIDLEHPLVFRGPNNSFFGYSVLEHYHDNTRWVIVGAPRANSTYSSSVHSPGAVYKCRVHSNPERRCTEMDLGRGNKPRESCGKTCQGDRDDEWMGVSLARQDRADGKILACAHRWKNVYYDSEHILPHGYCSIIPPTLQGRTKPLIPCYEGKENTKISRILRTDLLCMICLKSKLVVMGAPGSYYWTGTIKVYNLTSDSFYSPIKENIDSHRYSYLGYAVTAGHFSSPNVIDVAAGAPQHNGVGKVYIFKIDDGSLVKSFQDSGKMMGSYFGSSLCAVDLNQDGLSDLLVGAPMHSQLRDEGQVSVYLSKGNGVMEEQDVLTGDSSFNAHFGECITAIGDIDDDGYQDVAIGAPKEDDYAGAVYIYHGDPAGIISKYSMKLSGRSVNPGLQMFGQSISGNVDMDGNGYADVTIGAFMADSVVLLRSRPVITVDVSIFLPVSINISVPQCHEGHQNINCFNVSVCMRFRGRQLPGQIELLYNLTADVDKRQKSQPARVYFTQSGSQISQMSRQLSLDINREECQRYTAYVKKDVKEVFTAITFEVAYSLGKHVLTGHQERDLPALTPVLRWGKGKQIAVRNETWFEKNCLSDDCAADLRLHGKMLLSGKPHLALGGVKNVSLNLTISNAGDDAYDTNIYFNFSREVFYINFWQKEEKGISCGLVDLDFLKCSVGFPFMRAQTKYHFAVIFDTSQLSGENDTLQFLVQAKSVVTPSSFVYGNSIDASRFVQLEDMECNFQPLNLTFQAINKGPSRLPGSTVDIRIPNRLAGSGADMFHIIETQVADGRGNCTPHRNPTPCTIPQDRESIFHTIFAFFTKSGRKVLDCDRPGRACMTISCSLGPQLTEEALSIDIKLLLNTEILKRDSSSVIQFVTRGNVQVNDRTLEVPNGLPEDISLVFEALHSQEPRGYVVGWIIAISLLVGILIFLLLAVLLWKMGFFRRRYREIIEAEKNRKDSDESWDWMEKNH

In [23]:
def edit_distance(s1, s2):
    # source: https://stackoverflow.com/questions/2460177/edit-distance-in-python
    
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

edit_distance(knotted.loc['AF-A0A671YRA8-F1'].uniprotSequence, unknotted.loc['AF-A0A671YMH3-F1'].uniprotSequence)

24